In [ ]:
!pip install moviepy opencv-python numpy


In [ ]:
from google.colab import files
uploaded = files.upload()

IMAGE_PATHS = list(uploaded.keys())[:10]  # max 10 images


Saving 9.png to 9 (1).png
Saving 8.png to 8 (1).png
Saving 7.png to 7 (1).png
Saving 6.png to 6 (1).png
Saving 5.png to 5 (1).png
Saving 3.png to 3 (1).png
Saving 2.png to 2 (2).png
Saving 1.png to 1 (2).png


In [16]:
import os
import cv2
import numpy as np
from moviepy.editor import VideoClip, concatenate_videoclips, vfx

# =============================
# USER SETTINGS
# =============================
MAX_IMAGES = 10
MAX_TOTAL_DURATION = 360   # 6 minutes
FPS = 24
OUTPUT_SIZE = (1280, 720)

OUTPUT_DIR = "/content/documentary"
VIDEO_NAME = "documentary_smooth.mp4"

CROSSFADE = 2.5            # longer, smoother
ZOOM_RANGE = (1.03, 1.10)  # very gentle zoom
PAN_RANGE = 0.06           # very small pan

# =============================
# LOAD IMAGES IN ORDER
# =============================
def load_images():
    paths = []
    for i in range(1, MAX_IMAGES + 1):
        for ext in ["jpg", "png", "jpeg"]:
            p = f"{i}.{ext}"
            if os.path.exists(p):
                paths.append(p)
                break
    return paths

IMAGE_PATHS = load_images()
if not IMAGE_PATHS:
    raise ValueError("No images found")

# =============================
# TIME DISTRIBUTION
# =============================
num_images = len(IMAGE_PATHS)
clip_duration = min(MAX_TOTAL_DURATION / num_images, 50)

# =============================
# EASING FUNCTION (KEY)
# =============================
def ease_in_out(t):
    # cubic smoothstep
    return 3*t*t - 2*t*t*t

# =============================
# DOCUMENTARY CLIP
# =============================
def documentary_clip(image_path, duration, index):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]

    zoom_start = ZOOM_RANGE[0]
    zoom_end   = ZOOM_RANGE[1]

    pan_x = PAN_RANGE if index % 2 == 0 else -PAN_RANGE
    pan_y = PAN_RANGE/2 if index % 3 == 0 else -PAN_RANGE/2

    def make_frame(t):
        p = ease_in_out(min(t / duration, 1.0))
        scale = zoom_start + (zoom_end - zoom_start) * p

        nw, nh = int(w * scale), int(h * scale)
        resized = cv2.resize(img, (nw, nh))

        cx = int((nw - OUTPUT_SIZE[0]) / 2 + pan_x * w * p)
        cy = int((nh - OUTPUT_SIZE[1]) / 2 + pan_y * h * p)

        cx = max(0, min(cx, nw - OUTPUT_SIZE[0]))
        cy = max(0, min(cy, nh - OUTPUT_SIZE[1]))

        return resized[cy:cy+OUTPUT_SIZE[1], cx:cx+OUTPUT_SIZE[0]]

    clip = VideoClip(make_frame, duration=duration)
    return clip.fx(vfx.fadein, 1.2).fx(vfx.fadeout, 1.2)

# =============================
# BUILD FINAL DOCUMENTARY
# =============================
clips = [
    documentary_clip(path, clip_duration, i)
    for i, path in enumerate(IMAGE_PATHS)
]

final_video = concatenate_videoclips(
    clips,
    method="compose",
    padding=-CROSSFADE
)

# =============================
# EXPORT
# =============================
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PATH = os.path.join(OUTPUT_DIR, VIDEO_NAME)

final_video.write_videofile(
    OUTPUT_PATH,
    fps=FPS,
    codec="libx264",
    audio=False
)

print("✅ Ultra-smooth documentary video saved at:")
print(OUTPUT_PATH)


Found 8 images
Creating cinematic clips...
  Processing image 1/8: 1.png
  Processing image 2/8: 2.png
  Processing image 3/8: 3.png
  Processing image 4/8: 5.png
  Processing image 5/8: 6.png
  Processing image 6/8: 7.png
  Processing image 7/8: 8.png
  Processing image 8/8: 9.png
Adding smooth transitions...
Rendering final video...
Moviepy - Building video /content/documentary/documentary_cinematic.mp4.
Moviepy - Writing video /content/documentary/documentary_cinematic.mp4



Moviepy - Done !
Moviepy - video ready /content/documentary/documentary_cinematic.mp4

✅ CINEMATIC DOCUMENTARY CREATED SUCCESSFULLY!
Output: /content/documentary/documentary_cinematic.mp4
Duration: 338.9 seconds
Resolution: 1920x1080
FPS: 30


In [17]:
from google.colab import files
files.download("/content/documentary/documentary_smooth.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>